In [1]:
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.types import timeseries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import matplotlib.pyplot as plt
from numpy.random import uniform, randint
import pylab
from pycbc.filter import sigma
from pycbc.waveform import get_td_waveform
from pycbc.psd import interpolate
from pycbc.types.timeseries import load_timeseries

#import false signal functions
from ipynb.fs.full.Falsig_functions import random_false_sig, flip_gw

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
# Generate 1 seconds of noise at 4096 Hz, randomized
# argument:
# sig: timeseries
# snr: desired signal-noise ratio
# output noise is scaled to desired snr

def noise(sig,snr):
    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    # Scale
    new_psd = pycbc.psd.estimate.interpolate(psd, sig.delta_f)  # First interpolate psd to desire delta_f
    current_snr = sigma(sig, psd=new_psd, low_frequency_cutoff=20.0)
    c = current_snr/snr
    scaled_ts = c * ts

    return scaled_ts

In [3]:
# Generate gravitational waveform with specified mass parameters.
# Other arguments fixed
# return the plus polarization

# For the ease of merging, the waveform is shifted and its start_time is reset to 0.

def gw(m1,m2):
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    # resize to window of 1 second
    hp.resize(4096)
    
    # randomly shift
    # To fit the whole waveform in the 1 second window, need to specify a cyclic shift range. 
    # 0.05 act as buffer since signal still oscillates a little after merging.
    shift_range = 1 - abs(hp.start_time)-0.05 
    shifted = hp.cyclic_time_shift(uniform(0.0,shift_range))

    # reset start time
    shifted.start_time = 0
    
    return shifted
    

In [4]:
def burried_gw(m1,m2,snr):
    sig = gw(m1,m2)
    noi = noise(sig,snr)
    
    return noi + sig

In [5]:
def burried_false(mag,snr):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise(fal,snr)
    
    return noi + fal

In [6]:
def burried_flip(m1,m2,snr):
    hp = gw(m1,m2)
    flipped = flip_gw(hp)
    noi = noise(hp,snr)
    
    return noi + flipped

In [7]:
# A crude visualization because not sure how to tune parameters for qtransform.
# Show the plot

def visualize(wave):
    times, freqs, power = wave.qtransform(.001, logfsteps=100,
                                                qrange=(4, 4),
                                                frange=(20, 512),)
    pylab.figure(figsize=[15, 3])
    pylab.pcolormesh(times, freqs, power**0.5)
    pylab.yscale('log')
    pylab.show()

In [12]:
'''
# test
test1 = burried_gw(30,35,1000)
test2 = burried_flip(30,35,1000)
test3 = burried_false(-20,1000)

# plot
pylab.plot(test1.sample_times, test1)
pylab.show()
# plot
pylab.plot(test2.sample_times, test2)
pylab.show()
# plot
pylab.plot(test3.sample_times, test3)
pylab.show()

visualize(test1)
'''

'\n# test\ntest1 = burried_gw(30,35,1000)\ntest2 = burried_flip(30,35,1000)\ntest3 = burried_false(-20,1000)\n\n# plot\npylab.plot(test1.sample_times, test1)\npylab.show()\n# plot\npylab.plot(test2.sample_times, test2)\npylab.show()\n# plot\npylab.plot(test3.sample_times, test3)\npylab.show()\n\nvisualize(test1)\n'